In [40]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [41]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '03838c6d9170b5d98f337aa01b64686b', 'Date': 'Fri, 23 Aug 2024 22:28:42 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [42]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Emily Johnson',
  'subject': 'Biology',
  'stars': 4,
  'review': 'Engaging lectures and fair grading. Could improve on office hours availability.'},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Computer Science',
  'stars': 5,
  'review': 'Excellent teacher! Makes complex topics easy to understand. Always willing to help.'},
 {'professor': 'Dr. Sarah Williams',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Knowledgeable but lectures can be dry. Assignments are challenging.'},
 {'professor': 'Prof. Robert Taylor',
  'subject': 'Mathematics',
  'stars': 4,
  'review': 'Clear explanations and helpful practice problems. Sometimes moves too quickly.'},
 {'professor': 'Dr. Lisa Garcia',
  'subject': 'Chemistry',
  'stars': 5,
  'review': 'Passionate about the subject. Lab sessions are well-organized and informative.'},
 {'professor': 'Prof. David Brown',
  'subject': 'History',
  'stars': 2,
  'review': 'Lectures are disorganized. Grading seems inconsistent

In [43]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id": review['professor'],
        "metadata": {
            "review": review["review"],
            "subject": review["subject"],
            "stars": review["stars"]
        }
    })

In [44]:
processed_data[0]

{'values': [-0.044517085,
  0.040580202,
  0.03364247,
  0.035294313,
  0.03259631,
  0.018624496,
  0.017922463,
  0.027640788,
  0.019450415,
  0.033725064,
  0.020868244,
  -0.01130822,
  0.0033363726,
  -0.016903829,
  0.0017464765,
  -0.006466265,
  0.005309977,
  0.011783125,
  0.03642307,
  0.011790007,
  0.028659422,
  -0.032045692,
  0.058640316,
  0.038377747,
  -0.029182505,
  -0.01843178,
  0.0071029114,
  -0.028218932,
  -0.0063802316,
  0.026181662,
  0.07648019,
  -0.021983236,
  -0.013008239,
  -0.01677994,
  -0.057428967,
  0.09134675,
  0.03735911,
  0.037386645,
  0.04049761,
  -0.020882009,
  0.001997694,
  -0.0018978952,
  0.008555154,
  -0.03226594,
  0.008919936,
  -0.024557354,
  0.006301081,
  -0.012402564,
  0.053739857,
  0.029292628,
  -0.0240618,
  0.0021456713,
  0.06348571,
  -0.05913587,
  0.005898445,
  -0.014605017,
  -0.013772215,
  -0.016683582,
  0.018115178,
  0.009794034,
  0.04638917,
  -0.018046351,
  0.021694165,
  0.006748454,
  -0.020097386,


In [45]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 20}

In [48]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 23}},
 'total_vector_count': 23}